# Importsand preparations

In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, DatasetDict

/itf-fi-ml/home/krimhau/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.1.0+cu121
CUDA version: 12.1
cuDNN version: 8902
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.35.0
Datasets version: 2.14.6


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Wed Feb 14 08:34:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
| 22%   26C    P8              14W / 250W |  10503MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 93% |
|  1 | 68% | 72% |
|  2 | 67% | 72% |
|  3 |  0% | 93% |
|  4 | 66% | 72% |
|  5 |  0% | 90% |
|  6 | 67% | 72% |
|  7 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% | 93% |
|  1 | 68% | 72% |
|  2 | 67% | 72% |
|  3 |  0% | 93% |
|  4 | 66% | 72% |
|  5 |  0% | 90% |
|  6 | 67% | 72% |
|  7 |  0% |  1% |


In [8]:
# Smaller and faster than bert.
model_ckpt = "distilbert-base-uncased"

epochs = 5 #Number of full cycles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evaluation examples in on iteration.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

Load dataset from huggingface

In [9]:
dataset = load_dataset("kristmh/high_vs_random_with_med_low_min_100_issues_per_repo")
dataset

Generating validate split: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11675/11675 [00:00<00:00, 89346.84 examples/s]


DatasetDict({
    test: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__'],
        num_rows: 11676
    })
    train: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__'],
        num_rows: 93401
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__'],
        num_rows: 11675
    })
})

In [10]:
test_dataset = load_dataset(("kristmh/test_high_vs_med_low_priority"))
test_dataset

DatasetDict({
    test_separate: Dataset({
        features: ['text_clean', 'labels', 'class'],
        num_rows: 22097
    })
})

In [11]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

## Tokenization

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    Tokenizing the whole dataset

In [15]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

In [16]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11675/11675 [00:12<00:00, 932.89 examples/s]


DatasetDict({
    test: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 11676
    })
    train: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 93401
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 11675
    })
})

In [17]:
test_tokenized_dataset = test_dataset.map(tokenize, batched=True)
test_tokenized_dataset

DatasetDict({
    test_separate: Dataset({
        features: ['text_clean', 'labels', 'class', 'input_ids', 'attention_mask'],
        num_rows: 22097
    })
})

In [18]:
separate_test_set = test_tokenized_dataset["test_separate"]
separate_test_set = separate_test_set.remove_columns(["text_clean", "class"])
separate_test_set

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 22097
})

In [19]:
# List the columns of the dataset.
# Should be: ["attention_mask", "input_ids", "labels", "token_type_ids"]
# Remove unnecessary columns that the model does not expect.
tokenized_dataset = tokenized_dataset.remove_columns(["text_clean", "__index_level_0__"])

train_dataset = tokenized_dataset["train"]
print(train_dataset)
validation_dataset = tokenized_dataset["validate"]
print(validation_dataset)
test_dataset = tokenized_dataset["test"]
test_dataset

Dataset({
    features: ['Unnamed: 0', 'label', 'input_ids', 'attention_mask'],
    num_rows: 93401
})
Dataset({
    features: ['Unnamed: 0', 'label', 'input_ids', 'attention_mask'],
    num_rows: 11675
})


Dataset({
    features: ['Unnamed: 0', 'label', 'input_ids', 'attention_mask'],
    num_rows: 11676
})

## Training a classifier

In [20]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [22]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.626200,0.605034,0.676146,[0.66352229 0.68785602],[0.687696 0.66613367],[0.64099037 0.71104284]
2,0.592600,0.596781,0.691306,[0.69601889 0.6864451 ],[0.68311258 0.70008873],[0.70942228 0.67332309]
3,0.519100,0.594909,0.694218,[0.71426285 0.67114959],[0.66816412 0.72903742],[0.76719395 0.62177846]
4,0.468300,0.611659,0.695846,[0.71833109 0.66945918],[0.66676484 0.73628174],[0.77854195 0.61375661]
5,0.429900,0.666898,0.694818,[0.70355271 0.68555291],[0.68160567 0.70979532],[0.72696011 0.66291176]


TrainOutput(global_step=29190, training_loss=0.5180433162886543, metrics={'train_runtime': 9026.733, 'train_samples_per_second': 51.736, 'train_steps_per_second': 3.234, 'total_flos': 6.186293750934528e+16, 'train_loss': 0.5180433162886543, 'epoch': 5.0})

* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, validation loss increases = Overfitting

In [23]:
# Evaluate validation set
eval_result = trainer.evaluate(eval_dataset=validation_dataset)

In [24]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.6948179871520342

eval_f1 = [0.70355271 0.68555291]

eval_loss = 0.6668977737426758

eval_precision = [0.68160567 0.70979532]

eval_recall = [0.72696011 0.66291176]

eval_runtime = 72.508

eval_samples_per_second = 161.017

eval_steps_per_second = 5.034



In [25]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [26]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.6959575196985269

eval_f1 = [0.70597979 0.68522788]

eval_loss = 0.6690625548362732

eval_precision = [0.68465863 0.70886076]

eval_recall = [0.72867157 0.66311996]

eval_runtime = 72.5079

eval_samples_per_second = 161.031

eval_steps_per_second = 5.034



In [27]:
trainer.save_model(model_dir + "_local_min_len_100") 

In [28]:
# Evaluate separate test set
separate_test_results = trainer.evaluate(eval_dataset=separate_test_set)

In [29]:
for key, value in sorted(separate_test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.5986332986378241

eval_f1 = [0.62174265 0.57251651]

eval_loss = 0.8498427867889404

eval_precision = [0.5890101  0.61088253]

eval_recall = [0.65832731 0.53868481]

eval_runtime = 137.4307

eval_samples_per_second = 160.786

eval_steps_per_second = 5.028



In [30]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local_min_len_100")

In [31]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [32]:
classifier("this does not need to be done fast")

[{'label': 'LABEL_0', 'score': 0.9496676325798035}]

In [33]:
classifier("this is super important")

[{'label': 'LABEL_0', 'score': 0.9207320213317871}]

In [34]:
classifier("this bug has super high impact on the project")

[{'label': 'LABEL_0', 'score': 0.8633365035057068}]

## Important to delete large objects to free memory 
del train_dataset

In [35]:
del validation_dataset 

In [36]:
del model

In [37]:
# Free cache
torch.cuda.empty_cache()

In [38]:
!nvidia-smi

Wed Feb 14 11:11:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
| 22%   26C    P8              13W / 250W |  10503MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--